In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

print(os.listdir("../data"))
PREC_PATH = "../data/preprocessed/{}.csv"
PATH = "../data/{}.csv"

games = pd.read_csv(PREC_PATH.format("games"))
details = pd.read_csv(PREC_PATH.format("details"))
players = pd.read_csv(PATH.format("players"))
ranking = pd.read_csv(PREC_PATH.format("rankings"))
teams = pd.read_csv(PATH.format("teams"))

['.gitattributes', 'games.csv', 'games_details.csv', 'map', 'players.csv', 'preprocessed', 'ranking.csv', 'teams.csv']


C:\Users\mathi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
player_selection = details.groupby("player_name").mean()[["player_id", "pts"]].reset_index()\
    .merge(players, right_on="PLAYER_ID", left_on="player_id")\
    .sort_values("pts", ascending=False)\
    .drop_duplicates("player_name")\
    .head(100)\
    .merge(teams[["TEAM_ID", "ABBREVIATION"]], on="TEAM_ID")\
    [["player_name", "player_id", "pts"]]


C:\Users\mathi\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:1204: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(


In [5]:
player_selection.head()

,player_name,player_id,pts
0,Kevin Durant,201142.0,26.958736
1,Russell Westbrook,201566.0,22.543830
2,LeBron James,2544.0,26.825015
3,Kyrie Irving,202681.0,22.660485
4,Collin Sexton,1629012.0,19.615721


In [4]:
#player_selection.to_csv("../website/data/player_selection.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../website/data/player_selection.csv'

In [15]:
os.listdir("../..")

['.idea',
 'datavis-project-2022-lebron-jenkins',
 'dataviz-exercise-02',
 'dataviz-exercise-03',
 'DataVizExercise01',
 'idea.txt',
 'intro.html']

# Teams

Take the teamId the name and the abreviation

In [3]:
coordinates = [(1610612737, 33.75725092895011, -84.39619973311765),
 (1610612738, 42.36610686544133, -71.06264504509178),
 (1610612740, 29.950485009327856, -90.08072646871527),
 (1610612741, 41.881968536545465, -87.6728455633052),
 (1610612742, 32.789894151360954, -96.8092592302266),
 (1610612743, 39.74958004618605, -105.00661578713809),
 (1610612745,29.750871888172558, -95.36167139097245 ),
 (1610612746, 34.04276826661928, -118.26669624321944),
 (1610612747, 34.04276826661928, -118.26669624321944),
 (1610612748, 25.7808987709237, -80.18623948495552),
 (1610612749, 43.045205319315365, -87.91665667747223),
 (1610612750, 44.979584391924476, -93.27553683777344),
 (1610612751, 40.682646169430356, -73.9755443855573),
 (1610612752, 40.75053668021922, -73.99313833210212),
 (1610612753, 28.539371922106323, -81.38441144428242),
 (1610612754, 39.76381215213423, -86.15605172410166),
 (1610612755, 39.90123409349528, -75.17090665637734),
 (1610612756, 33.44605897210844, -112.07107159689456),
 (1610612757, 45.53140421236139, -122.66711521703822),
 (1610612758, 38.58017062553036, -121.50008939400314),
 (1610612759, 29.427019813868597, -98.43815188998576),
 (1610612760, 35.463704015112775, -97.51550008012),
 (1610612761, 43.643465766008646, -79.37961392206583),
 (1610612762, 40.76774773159542, -111.90147367756563),
 (1610612763, 35.13775544220234, -90.05110142629135),
 (1610612764, 38.8980001758342, -77.0211572478214),
 (1610612765, 42.34072161638615, -83.0549240159282),
 (1610612766, 35.22455561049105, -80.83926239612208),
 (1610612739, 41.49641849438575, -81.68857242321435),
 (1610612744, 37.76801468162943, -122.38730084231904)]

df_coord = pd.DataFrame(coordinates,columns =['TEAM_ID', 'LATITUDE', 'LONGITUDE'])

In [4]:
teams.head()

,LEAGUE_ID,TEAM_ID,MIN_YEAR,MAX_YEAR,ABBREVIATION,NICKNAME,YEARFOUNDED,CITY,ARENA,ARENACAPACITY,OWNER,GENERALMANAGER,HEADCOACH,DLEAGUEAFFILIATION
0,0,1610612737,1949,2019,ATL,Hawks,1949,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Lloyd Pierce,Erie Bayhawks
1,0,1610612738,1946,2019,BOS,Celtics,1946,Boston,TD Garden,18624.0,Wyc Grousbeck,Danny Ainge,Brad Stevens,Maine Red Claws
2,0,1610612740,2002,2019,NOP,Pelicans,2002,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Alvin Gentry,No Affiliate
3,0,1610612741,1966,2019,CHI,Bulls,1966,Chicago,United Center,21711.0,Jerry Reinsdorf,Gar Forman,Jim Boylen,Windy City Bulls
4,0,1610612742,1980,2019,DAL,Mavericks,1980,Dallas,American Airlines Center,19200.0,Mark Cuban,Donnie Nelson,Rick Carlisle,Texas Legends


In [12]:
teams_summary = teams[['TEAM_ID','ABBREVIATION', 'NICKNAME','CITY']].merge(df_coord, right_on="TEAM_ID", left_on="TEAM_ID")
teams_summary = teams_summary.sort_values(by=['CITY'])
teams_summary.to_csv("../data_web/teams_summary.csv")

In [13]:
teams_summary.head()

,TEAM_ID,ABBREVIATION,NICKNAME,CITY,LATITUDE,LONGITUDE
0,1610612737,ATL,Hawks,Atlanta,33.757251,-84.396200
1,1610612738,BOS,Celtics,Boston,42.366107,-71.062645
12,1610612751,BKN,Nets,Brooklyn,40.682646,-73.975544
27,1610612766,CHA,Hornets,Charlotte,35.224556,-80.839262
3,1610612741,CHI,Bulls,Chicago,41.881969,-87.672846


# Games

## Get the season of one team with all its matches

In [48]:
import numpy as np
def get_games_summary_for_all_teams():
    games_summary = games[["home_team_id", "visitor_team_id", "home_team_wins", "season", "date", "pts_home", "pts_away"]]
    games_home_teams_summary = games_summary.copy(deep=True)
    games_home_teams_summary.rename(columns = {'home_team_id':'team', 'visitor_team_id':'adversary', 'home_team_wins':'team_wins'}, inplace = True)
    games_home_teams_summary["home"] = 1
    games_home_teams_summary = games_home_teams_summary[['team','adversary','home','team_wins','season','date','pts_home','pts_away']]

    games_away_teams_summary = games_summary.copy(deep=True)
    games_away_teams_summary.rename(columns = {'home_team_id':'adversary', 'visitor_team_id':'team', 'home_team_wins':'team_wins'}, inplace = True)
    games_away_teams_summary['team_wins'] = np.where(games_away_teams_summary['team_wins']==1, 0, 1)
    games_away_teams_summary["home"] = 0
    games_away_teams_summary = games_away_teams_summary[['team','adversary','home','team_wins','season','date','pts_home','pts_away']]

    return pd.concat([games_home_teams_summary,games_away_teams_summary])

def get_season(season, team_id):
    return games_summary.loc[(games_summary["team"] == team_id) & (games_summary["season"] == season)].sort_values(by=["date"])

games_summary = get_games_summary_for_all_teams()
sea = get_season(2003, 1610612748).copy(deep=True)

In [49]:
len(games_summary)

51394

In [50]:
# get win ratio from ranking
games_ranking = ranking.groupby(["team_id", "standingsdate"])[["g", "w", "w_pct"]].mean().reset_index()
games_summary = pd.merge(left=games_summary,
                         left_on=["team", "date"],
                         right=games_ranking, 
                         right_on=["team_id", "standingsdate"],
                         how="left")
len(games_summary)

51394

In [51]:
games_summary.head()

,team,adversary,home,team_wins,season,date,pts_home,pts_away,team_id,standingsdate,g,w,w_pct
0,1610612748,1610612750,1,0,2021,2022-03-12,104.0,113.0,1610612748,2022-03-12,69.0,45.0,0.652
1,1610612741,1610612739,1,1,2021,2022-03-12,101.0,91.0,1610612741,2022-03-12,67.0,41.0,0.612
2,1610612759,1610612754,1,0,2021,2022-03-12,108.0,119.0,1610612759,2022-03-12,68.0,26.0,0.382
3,1610612744,1610612749,1,1,2021,2022-03-12,122.0,109.0,1610612744,2022-03-12,68.0,46.0,0.676
4,1610612743,1610612761,1,0,2021,2022-03-12,115.0,127.0,1610612743,2022-03-12,68.0,40.0,0.588


In [54]:
import math
R = 6373.0

def get_coord_from_id(id):
    row = teams_summary[teams_summary["TEAM_ID"] == id]
    return [row["LATITUDE"],row["LONGITUDE"]]

def distance(coord1, coord2):
    lat1, lat2 = math.radians(coord1[0]), math.radians(coord2[0])
    dlon = math.radians(coord2[1]) - math.radians(coord1[1])
    dlat = lat2-lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    return R * c
    
def get_dist_for_match(entry):
    if entry["team"] == entry["team_"] and entry["season"] == entry["season_"] :
        last_coords = get_coord_from_id(entry["team"] if entry["home"] else entry["adversary"])
        new_coords = get_coord_from_id(entry["team"] if entry["home_"] else entry["adversary_"])
        return distance(last_coords,new_coords)
    else :
        return 0


games_summary_sorted = games_summary.sort_values(by=["team", "season", "date"])

x = games_summary_sorted.iloc[1:].reset_index()[["team", "adversary", "home", "season","date"]]
x.columns = [ c + "_" for c in x.columns]
concated_games_summary = pd.concat([games_summary_sorted.reset_index(),x], axis=1)
concated_games_summary["dist_for_this_match"] = concated_games_summary.apply(get_dist_for_match,axis =1)
concated_games_summary = concated_games_summary.drop(["team_", "adversary_", "home_", "season_", "date_"], axis=1)
concated_games_summary["game_loc_lat"] = concated_games_summary.apply(lambda entry : float(get_coord_from_id(entry["team"] if entry["home"] else entry["adversary"])[0]),axis =1)
concated_games_summary["game_loc_long"] = concated_games_summary.apply(lambda entry : float(get_coord_from_id(entry["team"] if entry["home"] else entry["adversary"])[1]),axis =1)
concated_games_summary.to_csv("../data_web/seasons.csv")

In [53]:
concated_games_summary.head(10)

,index,team,adversary,home,team_wins,season,date,pts_home,pts_away,team_id,standingsdate,g,w,w_pct,dist_for_this_match,game_loc_lat,game_loc_long
0,18320,1610612737,1610612739,1,0,2003,2003-10-08,80.0,83.0,1610612737,2003-10-08,0.0,0.0,0.000,683.690386,33.757251,-84.396200
1,44012,1610612737,1610612740,0,0,2003,2003-10-29,88.0,83.0,1610612737,2003-10-29,1.0,0.0,0.000,1344.581782,29.950485,-90.080726
2,43993,1610612737,1610612741,0,0,2003,2003-10-31,100.0,94.0,1610612737,2003-10-31,2.0,0.0,0.000,948.252786,41.881969,-87.672846
3,18284,1610612737,1610612754,1,0,2003,2003-11-01,99.0,103.0,1610612737,2003-11-01,3.0,0.0,0.000,0.000000,33.757251,-84.396200
4,18273,1610612737,1610612740,1,1,2003,2003-11-03,90.0,80.0,1610612737,2003-11-03,4.0,1.0,0.250,3434.201564,33.757251,-84.396200
5,43961,1610612737,1610612744,0,0,2003,2003-11-05,99.0,72.0,1610612737,2003-11-05,5.0,1.0,0.200,863.832017,37.768015,-122.387301
6,43937,1610612737,1610612757,0,0,2003,2003-11-08,90.0,83.0,1610612737,2003-11-08,6.0,1.0,0.167,2389.921630,45.531404,-122.667115
7,43931,1610612737,1610612760,0,1,2003,2003-11-09,81.0,91.0,1610612737,2003-11-09,7.0,2.0,0.286,1899.483236,35.463704,-97.515500
8,43915,1610612737,1610612746,0,0,2003,2003-11-11,115.0,103.0,1610612737,2003-11-11,8.0,2.0,0.250,576.774181,34.042768,-118.266696
9,43907,1610612737,1610612756,0,1,2003,2003-11-12,94.0,99.0,1610612737,2003-11-12,9.0,3.0,0.333,2556.383769,33.446059,-112.071072


In [55]:
teams_summary[teams_summary["TEAM_ID"] == 1610612755]

,TEAM_ID,ABBREVIATION,NICKNAME,LATITUDE,LONGITUDE
16,1610612755,PHI,76ers,39.901234,-75.170907


In [56]:
sea.to_csv("../data_web/season_heat_2003.csv")

In [40]:
sea

,team,adversary,home,team_wins,season,date,pts_home,pts_away
18330,1610612748,1610612755,1,1,2003,2003-10-07,86.0,79.0
18317,1610612748,1610612755,0,0,2003,2003-10-28,89.0,74.0
18307,1610612748,1610612738,0,0,2003,2003-10-29,98.0,75.0
18295,1610612748,1610612765,1,0,2003,2003-10-31,81.0,93.0
18274,1610612748,1610612742,0,0,2003,2003-11-03,103.0,93.0
...,...,...,...,...,...,...,...,...
17084,1610612748,1610612754,0,0,2003,2004-05-08,91.0,80.0
17080,1610612748,1610612754,1,1,2003,2004-05-10,94.0,87.0
17076,1610612748,1610612754,1,1,2003,2004-05-12,100.0,88.0
17071,1610612748,1610612754,0,0,2003,2004-05-15,94.0,83.0
